In [2]:
import BSE as bse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import itertools
import random

random.seed(42)
np.random.seed(42)

sess_id = "q3_demo_default"
start_time = 0
end_time = 60 * 10

buyers_spec = [('ZIP', 5), ('ZIC', 5)]
sellers_spec = [('ZIP', 5), ('SHVR', 5)]
mrktmakers_spec = [
    ('MMM01', 1, {})
]

traders_spec = {
    'buyers': buyers_spec,
    'sellers': sellers_spec,
    'mrktmakers': mrktmakers_spec
}

sup_range = (100, 200)
dem_range = (100, 200)

supply_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}
]
demand_schedule = [
    {'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}
]

order_schedule = {
    'sup': supply_schedule,
    'dem': demand_schedule,
    'interval': 30,
    'timemode': 'drip-jitter'
}

dumpfile_flags = {
    'dump_strats': False,
    'dump_lobs': False,
    'dump_tape': False,
    'dump_blotters': False,
    'dump_avgbals': True
}

bse.market_session(
    sess_id=sess_id,
    starttime=start_time,
    endtime=end_time,
    trader_spec=traders_spec,
    order_schedule=order_schedule,
    dumpfile_flags=dumpfile_flags,
    sess_vrbs=False,  
)

print(f"Session {sess_id} finished. Check {sess_id}_avg_balance.csv")


MM01 init: n_past_trades=1, bid_percent=0.50000, ask_delta=25

Session q3_demo_default finished. Check q3_demo_default_avg_balance.csv
